In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\mlops-youtube\\indi\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

# El decorador @dataclass automáticamente genera métodos especiales como __init__ y __repr__
# frozen=True hace que la instancia de la clase sea inmutable, es decir, sus atributos no pueden ser modificados después de la creación.
@dataclass(frozen=True)
class DataIngestionConfig:
    # root_dir: Path - Directorio raíz donde se almacenarán los artefactos de la ingesta de datos
    root_dir: Path
    
    # source_URL: str - URL de la fuente desde donde se descargará el archivo de datos
    source_URL: str
    
    # local_data_file: Path - Ruta local donde se almacenará el archivo de datos descargado
    local_data_file: Path
    
    # unzip_dir: Path - Directorio donde se descomprimirá el archivo de datos
    unzip_dir: Path


In [5]:
from src.indi.constants import *
from src.indi.utils.common import read_yaml, create_directories


In [11]:
# Definición de la clase ConfigurationManager que gestiona la carga de configuraciones, parámetros y esquemas.
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        # El constructor de la clase inicializa las rutas de los archivos de configuración,
        # parámetros y esquemas utilizando valores por defecto definidos anteriormente.
        
        # Lee el archivo de configuración y almacena el contenido en self.config.
        self.config = read_yaml(config_filepath)
        
        # Lee el archivo de parámetros y almacena el contenido en self.params.
        self.params = read_yaml(params_filepath)
        
        # Lee el archivo de esquema y almacena el contenido en self.schema.
        self.schema = read_yaml(schema_filepath)
        
        # Crea los directorios necesarios para almacenar los artefactos, 
        # utilizando la ruta definida en el archivo de configuración.
        create_directories([self.config.artifacts_root])

        # Método para obtener la configuración de la ingesta de datos
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # Extrae la configuración de ingesta de datos del archivo de configuración principal
        config = self.config.data_ingestion

        # Crea el directorio raíz de ingesta de datos si no existe
        create_directories([config.root_dir])

        # Crea una instancia de DataIngestionConfig con los valores extraídos de la configuración
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        # Devuelve la instancia de DataIngestionConfig
        return data_ingestion_config


# Funciones auxiliares utilizadas por ConfigurationManager (no definidas en el código proporcionado):
# - read_yaml(filepath): Lee un archivo YAML y devuelve su contenido como un diccionario u objeto.
# - create_directories(list_of_directories): Crea los directorios especificados si no existen.


In [8]:
import os
import urllib.request as request
import zipfile
from indi import logger
from indi.utils.common import get_size

In [9]:
import os
from pathlib import Path
from urllib import request
from indi import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Inicializa una instancia de DataIngestion con una configuración específica.

        Args:
            config (DataIngestionConfig): Objeto de configuración de ingesta de datos.
        """
        self.config = config

    def download_file(self):
        """
        Descarga el archivo desde la URL especificada en la configuración.
        Solo descarga el archivo si no existe localmente.
        """
        # Verifica si el archivo ya existe localmente
        if not os.path.exists(self.config.local_data_file):
            # Descarga el archivo desde la URL especificada
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            # Registra un mensaje de éxito con información adicional del encabezado de la descarga
            logger.info(f"{filename} descargado con la siguiente información: \n{headers}")
        else:
            # Si el archivo ya existe, registra un mensaje con el tamaño del archivo
            logger.info(f"El archivo ya existe con tamaño: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extrae el archivo zip en el directorio de datos especificado en la configuración.
        Crea el directorio de destino si no existe.
        """
        # Ruta del directorio donde se descomprimirá el archivo
        unzip_path = self.config.unzip_dir
        
        # Crea el directorio si no existe
        os.makedirs(unzip_path, exist_ok=True)
        
        # Abre el archivo zip y extrae su contenido en el directorio especificado
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        # Registro de éxito en la extracción
        logger.info(f"Archivo zip extraído en: {unzip_path}")

# Funciones y clases auxiliares (asumiendo que están definidas en otros lugares del proyecto)
# - DataIngestionConfig: Clase de configuración para la ingesta de datos
# - get_size(path): Función que obtiene el tamaño del archivo en KB y devuelve una cadena de texto con el tamaño aproximado
# - logger: Objeto de registro configurado para la aplicación


In [12]:
try:
    # Crear una instancia de ConfigurationManager para cargar las configuraciones
    config = ConfigurationManager()
    
    # Obtener la configuración de ingesta de datos
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Crear una instancia de DataIngestion con la configuración obtenida
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Descargar el archivo especificado en la configuración
    data_ingestion.download_file()
    
    # Extraer el archivo zip descargado en el directorio especificado
    data_ingestion.extract_zip_file()

except Exception as e:
    # Si ocurre alguna excepción, la re-lanza después de capturarla
    raise e


[2024-05-18 16:00:49,972:INFO:common:Archivo YAML: config\config.yaml cargado con éxito]
[2024-05-18 16:00:49,974:INFO:common:Archivo YAML: params.yaml cargado con éxito]
[2024-05-18 16:00:49,978:INFO:common:Archivo YAML: schema.yaml cargado con éxito]
[2024-05-18 16:00:49,981:INFO:common:Directorio creado en: artifacts]
[2024-05-18 16:00:49,982:INFO:common:Directorio creado en: artifacts/data_ingestion]


[2024-05-18 16:00:50,786:INFO:3949664249:artifacts/data_ingestion/data.zip descargado con la siguiente información: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4B12:7BC9:4A67DE:4FA4F3:6648B492
Accept-Ranges: bytes
Date: Sat, 18 May 2024 14:00:50 GMT
Via: 1.1 varnish
X-Served-By: cache-mad22041-MAD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1716040850.492978,VS0,VE179
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a1c0669e2fe37293f150a4fe9786699b8f6065a8
Expires: Sat, 18 May 2024 14:05:50 GMT
Source-Age: 0

]
[2024-05-18 16:00:50,818:INFO:3949664249

: 